In [17]:
#課題3

import numpy as np
import mnist
import matplotlib.pyplot as plt
from pylab import cm
import itertools

#乱数固定
np.random.seed(0)
#学習率
eta = 0.1
#エポック数
epoch = 5
#画像サイズ
size = 28*28
#中間層の数
M = 60
#画像枚数
train_data_size = 54000
test_data_size = 6000
#クラス数
classes = 10
#データ
X = mnist.download_and_parse_mnist_file("train-images-idx3-ubyte.gz")
Y = mnist.download_and_parse_mnist_file("train-labels-idx1-ubyte.gz")
#バッチサイズ
B = 100

#シグモイド関数
def sigmoid(x):
    s = 1.0 / (1.0 + np.exp(-x))
    return s
#ソフトマックス関数
def softmax(x):
    u = np.sum(np.exp(x),axis=0)
    return np.exp(x)/u



#乱数生成
W_1 = np.random.normal(0, np.sqrt(1/size),(M,size)) #M x size
b_1 = np.random.normal(0, np.sqrt(1/size),M) 
W_2 = np.random.normal(0, np.sqrt(1/M), (classes,M)) #classes x M
b_2 = np.random.normal(0, np.sqrt(1/M), classes)


def forward(W_1,W_2,b_1,b_2,data_size,test_flag):
    #ランダムなインデックス抽出
    minibach = np.random.choice(data_size, B, replace=False) + test_flag*train_data_size
    #size x Bの行列へ変換
    X_matrix = []
    for i in range(B):
        X_matrix.append(X[minibach[i]].reshape([size]))
    X_matrix = np.array(X_matrix).T #size x B
    X_matrix = X_matrix/255

    #中間層の出力
    y_1 = sigmoid(np.dot(W_1,X_matrix)+b_1[:,np.newaxis]) # M x B

    #ソフトマックス
    a = np.dot(W_2,y_1) + b_2[:,np.newaxis] # classes x B
    alpha = np.max(a,axis=0)#各列の最大値B個
    y_pred = softmax(a - alpha[np.newaxis,:]) #それぞれのバッチのソフトマックス #10 x B
    y_index_pred = np.argmax(y_pred,axis=0)  #それぞれのバッチの予測
    y_true = np.array([np.eye(10)[y] for y in Y[minibach]],dtype="int8").T  # 10 x B
    
    return(X_matrix, y_1, y_pred,y_index_pred, y_true, minibach)
    
def backward(y_pred, y_true, y_1, X_matrix, W_1, W_2,b_1, b_2):
    #中間層から出力層の逆伝播
    #del En/del akを求める
    del_en_del_ak = (y_pred - y_true)/B #C x B
    # print(del_en_del_ak)
    #del En/del X
    del_en_del_x = np.dot(W_2.T,del_en_del_ak) # M x B
    # print(del_en_del_x) 
    #del En/del W_2
    del_en_del_w_2 = np.dot(del_en_del_ak, y_1.T) #C x M
    #del En/del b_2
    del_en_del_b_2 = np.sum(del_en_del_ak,axis=1) #B
    # print(del_en_del_b_2)


    #sigmoidの逆伝搬
    #シグモイドの微分
    def del_sigmoid(y):
        return (1 - y)  * y
    
    #del En/del X
    del_en_del_x_1 = del_en_del_x * del_sigmoid(y_1)
    
    
    #全結合層１の逆伝搬
    #del En/del W
    del_en_del_w_1 = np.dot(del_en_del_x_1, X_matrix.T)
    #del En/del b
    del_en_del_b_1 = np.sum(del_en_del_x_1, axis=1)


    #パラメータの更新
    W_1 = W_1 - eta * del_en_del_w_1
    W_2 = W_2 - eta * del_en_del_w_2
    b_1 = b_1 - eta * del_en_del_b_1
    b_2 = b_2 - eta * del_en_del_b_2


    #クロスエントロピーの計算
    cross_entropy_list = -np.sum(y_true * np.log(y_pred + 1e-15),axis = 0)
    each_cross_entropy = np.sum(cross_entropy_list)/B
    return (each_cross_entropy,W_1,W_2,b_1,b_2)


#正解率
def list_match_rate(list1, list2):
    # 2つのリストの要素数を取得
    len1 = len(list1)
    len2 = len(list2)
    
    # 空のリストが渡された場合、一致率は0とします
    if len1 == 0 or len2 == 0:
        return 0.0
    
    # 一致する要素の数を計算
    match_count = sum(1 for a, b in zip(list1, list2) if a==b)
    
    # 一致率を計算
    match_rate = (match_count / len(list1)) * 100.0
    
    return match_rate

#epoch
j = 0 
while (epoch > 0):
    j += 1
    print("epoch" + str(j))
    sum_entropy = 0
#     W_1 = np.load("W1_test.npy")
#     W_2 = np.load("W2_test.npy")
#     b_1 = np.load("b1_test.npy")
#     b_2 = np.load("b2_test.npy")
    #繰り返し学習
    for i in range(int(train_data_size/B)):
        (X_matrix, y_1, y_pred,y_index_pred, y_true, minibach) = forward(W_1,W_2,b_1,b_2,train_data_size,0)
        (each_cross_entropy,W_1,W_2,b_1,b_2) = backward(y_pred, y_true, y_1, X_matrix, W_1, W_2,b_1, b_2)
        sum_entropy += each_cross_entropy

    cross_entropy = sum_entropy*B/train_data_size
    print("クロスエントロピー:"+ str(cross_entropy))
    print("正解率" + str(list_match_rate(Y[minibach], y_index_pred)))#最後の１００個のミニバッチの正解率
#     print("正しい結果: " + str(Y[minibach]))
#     print("予想結果: " + str(y_index_pred))
    
    #保存
#     np.save("/Users/jiho/OneDrive - Kyoto University/三回生後期/実験　画像/W1_test.npy",W_1)
#     np.save("/Users/jiho/OneDrive - Kyoto University/三回生後期/実験　画像/W2_test.npy",W_2)
#     np.save("/Users/jiho/OneDrive - Kyoto University/三回生後期/実験　画像/b1_test.npy",b_1)
#     np.save("/Users/jiho/OneDrive - Kyoto University/三回生後期/実験　画像/b2_test.npy",b_2)
    
    #テストデータで評価
    (X_matrix, y_1, y_pred,y_index_pred, y_true, minibach) = forward(W_1,W_2,b_1,b_2,test_data_size,1)
    print("テスト正解率" + str(list_match_rate(Y[minibach], y_index_pred)))
    
    
    epoch -= 1
    



epoch1
クロスエントロピー:1.0975279494508363
正解率85.0
テスト正解率84.0
epoch2
クロスエントロピー:0.4690466277687233
正解率90.0
テスト正解率85.0
epoch3
クロスエントロピー:0.37065750517502527
正解率91.0
テスト正解率81.0
epoch4
クロスエントロピー:0.3295653707361956
正解率94.0
テスト正解率90.0
epoch5
クロスエントロピー:0.30839350236442375
正解率93.0
テスト正解率94.0
